In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import random
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras import Model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow import keras
from tqdm import tqdm


print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

def seed_tensorflow(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

2024-07-28 14:20:49.420112: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 14:20:49.444740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 14:20:49.444768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 14:20:49.445359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 14:20:49.449651: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-28 14:20:50.456644: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:20:50.478843: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:20:50.478985: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
# binary_train_label
# short_play
# effective_play
# long_play
# complete_play
select_label = 'effective_play'

# base_model = "./model/kuairand_backbone/mlp_{}.h5".format(select_label)
CPF_model = "./model/kuairand_CPF/DCN_{}.h5".format(select_label)

In [3]:
para = {'data_path':'./kuai_input/',
        'embedding_dim':64,
        'seed':0,
        'lr':1e-5,
        'batch_size':512,
        'epochs':64,
        'verbose':1,
        'callback':{
                     'monitor':'val_auc',
                     'patience':10,
                     'CPF_model':CPF_model
                     },
        'mlp_dims':[256,128,64],
        'mlp_act':'relu',
        'mlp_dps':[.5,.5,.5],
       }


In [4]:
action = pd.read_csv(para['data_path']+"action.csv")
feed_emb = np.load(para['data_path']+"embeddings.npy")
action = action[['user_id','video_id','duration_level','tab','user_active_degree','author_id','music_id',
                 'short_play', 'binary_train_label', 'effective_play',
                 'long_play', 'complete_play']]
categories_to_keep = ['full_active', 'high_active', 'middle_active','low_active']

action = action[action['user_active_degree'].isin(categories_to_keep)]

category_mapping = {
    'low_active': 0,
    'middle_active': 1,
    'high_active': 2,
    'full_active': 3
}

action['user_active_degree_encoded'] = action['user_active_degree'].map(category_mapping)
action['music_id_encoded'], _ = pd.factorize(action['music_id'])

In [5]:
data_X = action[['user_id','video_id','duration_level','tab','user_active_degree_encoded','author_id','music_id_encoded','video_id']]#select_label]]
fields = data_X.max().values + 1 # 模型输入的feature_fields
fields

array([25492,  7015,     5,    15,     4,  6031,  6871,  7015])

In [6]:
train= action.groupby('user_id').apply(lambda x: x[:int(len(x)*0.6)]).reset_index(drop=True)
valid= action.groupby('user_id').apply(lambda x: x[int(len(x)*0.6):int(len(x)*0.8)]).reset_index(drop=True)
test= action.groupby('user_id').apply(lambda x: x[int(len(x)*0.8):]).reset_index(drop=True)

In [7]:
seed_tensorflow(seed=para['seed'])
"""
    uid_lay = get_layer((1,),'uid',d1 = max(action['user_id'])+1,d2 = para['embedding_dim'],trainable=True)
    videoid_lay = get_layer((1,),'video_id',d1 = max(action['video_id'])+1,d2 = int(para['embedding_dim']),trainable=True)
    duration_lay = get_layer((1,),'duration_id',d1 = max(action['duration_level'])+1,d2 = int(para['embedding_dim']),trainable=True)
    tab_lay = get_layer((1,),'tab',d1 = max(action['tab'])+1,d2 = int(para['embedding_dim']),trainable=True)
    user_active_degree_encoded = get_layer((1,),'user_active_degree_encoded',d1 = max(action['user_active_degree_encoded'])+1,d2 = int(para['embedding_dim']),trainable=True)
    author_lay = get_layer((1,),'author_id',d1 = max(action['author_id'])+1,d2 = int(para['embedding_dim']),trainable=True)
    music_lay = get_layer((1,),'music_id_encoded',d1 = max(action['music_id_encoded'])+1,d2 = int(para['embedding_dim']),trainable=True)
"""

features = ['user_id','video_id','duration_level','tab','user_active_degree_encoded','author_id','music_id_encoded','video_id',select_label]

# # short_play binary_train_label effective_play long_play complete_play
def get_input(df,is_test=False):
    X = []
    for f in features:
        X.append(df[f].values.reshape(-1,1))
    y = [df[select_label].values.reshape(-1,1)]
    return X,y

X_train,y_train = get_input(train,is_test=False)
X_valid,y_valid = get_input(valid,is_test=False)
X_test,y_test = get_input(test,is_test=True)
X_v,y_v = get_input(test[:4],is_test=True)

2024-07-28 14:21:03.591352: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:03.591516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:03.591583: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
train_X = np.hstack(X_train)
test_X = np.hstack(X_test)
valid_X = np.hstack(X_valid)

# 打印重构的数组以验证
print(train_X)

[[    0  1399     4 ...     0  1399     0]
 [    0  6857     2 ...     1  6857     0]
 [    0  5590     4 ...     2  5590     0]
 ...
 [25491  1799     2 ...   398  1799     0]
 [25491  1799     2 ...   398  1799     0]
 [25491  2426     0 ...  1069  2426     1]]


In [9]:
train_y = y_train[0].flatten()
test_y = y_test[0].flatten()
valid_y = y_valid[0].flatten()

In [10]:
train_X[:,-1:]

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [11]:
train_X[:,2:3]

array([[4],
       [2],
       [4],
       ...,
       [2],
       [2],
       [0]])

In [12]:
train_X.shape

(732591, 9)

In [13]:

class DeepCrossNet(Model):
    """
        Deep Cross Network
    """
    def __init__(self, feature_fields, embed_dim, num_layers, mlp_dims, dropout):

        super(DeepCrossNet, self).__init__()
        self.feature_fields = feature_fields
        self.offsets = np.array((0, *np.cumsum(feature_fields)[:-1]), dtype = np.int64)
        self.input_dims = sum(feature_fields[:-1]) + 1
        self.input_lens = len(feature_fields)


        #self.feature_fields = feature_fields
        #self.input_dims = sum(feature_fields[:-1]) + 1 + feed_emb.shape[0]# 前五个特征的总维度
        #self.input_lens = len(feature_fields)   # 前五个特征的数量



        #embedding
        self.embedding = layers.Embedding(self.input_dims,
                                          embed_dim,
                                          input_length = self.input_lens-1)

        self.pretrained_embedding = layers.Embedding(
                                              input_dim=feed_emb.shape[0],
                                              output_dim=feed_emb.shape[1],
                                              input_length=1,
                                              weights=[feed_emb] ,
                                              name='pretrain_embedding',
                                              trainable=False)  # 根据需要选择是否训练这层

        #DNN
        self.embed_out_dim = (len(feature_fields)-1) * embed_dim + feed_emb.shape[1]
        input_dim = self.embed_out_dim
        self.mlp = tf.keras.Sequential()

        self.mlp.add(layers.Dense(256, input_shape = [input_dim,]))
        self.mlp.add(layers.Activation('relu'))
        self.mlp.add(layers.Dropout(dropout))

        self.mlp.add(layers.Dense(128, input_shape = [256,]))
        self.mlp.add(layers.Activation('relu'))
        self.mlp.add(layers.Dropout(dropout))

        self.mlp.add(layers.Dense(64, input_shape = [input_dim,]))
        self.mlp.add(layers.Activation('relu'))
        #self.mlp.add(layers.Dropout(dropout))


        #Cross Net
        self.num_layers = num_layers

        #LR
        self.lr = layers.Dense(1, input_shape = [mlp_dims[-1]+self.embed_out_dim,])


    def build(self, input_shape):
        """
            自定义显式交叉层的参数
        """

        self.cross_w = [
            self.add_weight(name = 'w_{}'.format(i),
                            shape = (self.embed_out_dim, 1),
                            trainable = True,
                            initializer = 'random_normal'
                            )
            for i in range(self.num_layers)
        ]

        self.cross_b = [
            self.add_weight(name = 'b_{}'.format(i),
                            shape = (self.embed_out_dim,),
                            initializer = 'random_normal',
                            trainable = True)
            for i in range(self.num_layers)
        ]

    def call(self, x):
        #x = x + self.offsets
        # embedding
        #x = self.embedding(x)
        #x = tf.reshape(x, (-1, self.embed_out_dim))
        # = x + self.offsets
        # embedding
        custom_features = x[:, :-2] + self.offsets[:-1]
        pretrained_feature = x[:, -2:-1]

        custom_embed = self.embedding(custom_features)
        pretrained_embed = self.pretrained_embedding(pretrained_feature)
        #x = custom_embed
        x = layers.Concatenate(axis=1)([custom_embed, pretrained_embed])
        x = tf.reshape(x, (-1, self.embed_out_dim))

        #DNN out
        mlp_part = self.mlp(x)

        #Cross Net out
        x0 = x
        cross = x
        for i in range(self.num_layers):
            xw = tf.tensordot(cross, self.cross_w[i], axes = [1,0])
            cross = x0 * xw + self.cross_b[i] + cross

        #Concat output
        out = tf.concat([cross, mlp_part], axis = 1)
        #tf.print(out.shape)
        #LR predict
        #out = self.lr(out)
        #out = tf.sigmoid(out)

        return out




In [14]:
base_model_backbone = DeepCrossNet(feature_fields=fields, embed_dim=64, num_layers=3, mlp_dims=[256, 128,64], dropout=0.5)

2024-07-28 14:21:03.694669: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:03.694856: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:03.694925: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
from tensorflow.keras.layers import Input, Dense,Lambda
from tensorflow.keras.models import Model
inputs = Input(shape=(9,), dtype='int32')
duration_id_input = Lambda(lambda x: x[:, 2:3], name='duration_id_extractor')(inputs)
labels = Lambda(lambda x: x[:, -1:], name='labels')(inputs)

# 将输入层传递给自定义模型
outputs = base_model_backbone(inputs)
output = tf.keras.layers.Dense(1, activation='sigmoid')(outputs)
# 创建包含输入层和输出层的完整模型
model = Model(inputs=inputs, outputs=output)

# 查看模型结构
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 deep_cross_net (DeepCrossN  (None, 576)               3532288   
 et)                                                             
                                                                 
 dense_4 (Dense)             (None, 1)                 577       
                                                                 
Total params: 3532865 (13.48 MB)
Trainable params: 3083905 (11.76 MB)
Non-trainable params: 448960 (1.71 MB)
_________________________________________________________________


In [16]:
class Prototypes(tf.keras.layers.Layer):
    name = 'prototypes'
    def __init__(self,
                 k ,
                 beta1=0.0,
                 beta2=0.0,
                 beta3=0.0,
                 init_prototypes = None,
                 **kwargs):
        super(Prototypes, self).__init__(**kwargs)
        self.k = k
        self.beta1, self.beta2, self.beta3 = beta1, beta2, beta3
        # y_train[0]
        self.init_prototypes = init_prototypes
        self.binary_classifiers = []
        # 
        for _ in range(5):
            mlp = tf.keras.Sequential([
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
            mlp.build((None, 576))
            self.binary_classifiers.append(mlp)
            
    # define prototype
    def build(self, input_shape):
        self.prototypes = self.add_weight(
            name='prototypes',
            shape=(1, self.k, 576),
            initializer=tf.keras.initializers.Constant(self.init_prototypes[tf.newaxis, :, :]),
            trainable=True
        )
        super(Prototypes, self).build(input_shape)

    
    def orthogonality_loss(self,prototypes, k):
        # 确保传入的 k 是偶数
        assert k % 2 == 0, "k must be even."
        D = k // 2

        prototypes = tf.reshape(prototypes, [k, 576])
        prototypes = tf.nn.l2_normalize(prototypes, axis=1)

        cosine_similarity_matrix = tf.matmul(prototypes, prototypes, transpose_b=True)

        upper_triangular_part = tf.linalg.band_part(cosine_similarity_matrix, 0, -1)
        upper_triangular_part -= tf.linalg.band_part(cosine_similarity_matrix, 0, 0)  # 去掉对角线

        loss = tf.reduce_sum(tf.square(upper_triangular_part))
        normalization_factor = D * (2 * D - 1)
        loss /= normalization_factor

        return loss

    def contrastive_loss(self, vec, labels, duration):
        # Normalize the vectors
        vec_norm = tf.nn.l2_normalize(vec, axis=1)
        #print("vec_norm",vec_norm)
        sim_matrix = tf.matmul(vec_norm, vec_norm, transpose_b=True)
        #print("sim_matrix",sim_matrix)
        # Create masks
        label_eq = tf.equal(labels, tf.transpose(labels))
        #print("label_eq",label_eq)
        duration_eq = tf.equal(duration, tf.transpose(duration))
        #print("duration_eq",duration_eq)
        mask = tf.logical_and(label_eq, duration_eq)
        mask = tf.cast(mask, dtype=tf.float32)
        #print("mask",mask)

        # Compute the softmax denominator
        exp_sim = tf.exp(sim_matrix)
        total_sum = tf.reduce_sum(exp_sim)
        #print("total_sum",total_sum)
        normal_sim = exp_sim / total_sum
        #print("normal_sim",normal_sim)

        log_normal_sim = tf.math.log(normal_sim)
        weighted_log_prob = mask * log_normal_sim

        num_ones = tf.reduce_sum(mask)

        loss = -tf.reduce_sum(weighted_log_prob)
        return loss/num_ones

    def assign_loss(self,output, vec):


        output_norm = tf.nn.l2_normalize(output, axis=1)
        vec_norm = tf.nn.l2_normalize(vec, axis=1)

        cosine_similarity = tf.reduce_sum(tf.multiply(output_norm, vec_norm), axis=1)
        loss = 1 - tf.reduce_mean(cosine_similarity)
        return loss
    def get_alpha(self,n_batch,duration_one_hot_expanded,alpha_product):
        """
        alpha_product : (n,10,1)
        duration_one_hot_expanded: (n,5,1)
        """
        expanded_alpha = tf.reshape(alpha_product, [n_batch, 5, 2, 1])
        masked = expanded_alpha * tf.cast(duration_one_hot_expanded, tf.float32)[:, :, :, tf.newaxis]
        # n,2,1
        alpha_para = tf.reduce_sum(masked, axis=1)
        alpha_neg = alpha_para[:, 0, :]  # 第一个元素，保留最后一个轴
        alpha_pos = alpha_para[:, 1, :]  # 第二个元素，保留最后一个轴
        return alpha_neg,alpha_pos
    def mse_loss(self,vec,label):
        vec = tf.convert_to_tensor(vec, dtype=tf.float32)

        label = tf.convert_to_tensor(label, dtype=tf.float32)

        squeezed_label = tf.squeeze(label)

        #tf.print(label, summarize=-1)
        positive_indices = tf.where(squeezed_label == 1)
        negative_indices = tf.where(squeezed_label == 0)

        positive_samples = tf.gather(vec, positive_indices)
        negative_samples = tf.gather(vec, negative_indices)
        positive_samples = tf.squeeze(positive_samples,axis=1)
        negative_samples = tf.squeeze(negative_samples,axis=1)
        positive_loss = tf.reduce_mean(tf.square(positive_samples - self.pos_mean_prototype))
        negative_loss = tf.reduce_mean(tf.square(negative_samples - self.neg_mean_prototype))

        total_loss = positive_loss * 0.7 + negative_loss * 0.3

        return total_loss

    def binary_loss(self,y_true, y_pred):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        loss = bce(y_true, y_pred)
        return loss     
    
    def create_mask(self,duration, batch_size, num_classes=40, bucket_size=8):
        start_indices = duration * bucket_size
        masks = []
        for i in range(batch_size):
            mask = tf.concat([
                tf.zeros((start_indices[i, 0], 1), dtype=tf.float32),
                tf.ones((bucket_size, 1), dtype=tf.float32),
                tf.zeros((num_classes - start_indices[i, 0] - bucket_size, 1), dtype=tf.float32)
            ], axis=0)
            masks.append(mask)
        return tf.stack(masks)
    def binary_crossentropy_manual(self, y_true, y_pred,para):
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

        # 防止 log(0) 的情况发生
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        loss = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)

        weights = tf.where(y_true == 1, para, 1.0)
        weighted_loss = loss * weights

        positive_loss = tf.boolean_mask(weighted_loss, y_true == 1)
        negative_loss = tf.boolean_mask(weighted_loss, y_true == 0)
        #total_positive_loss = tf.reduce_sum(positive_loss)
        #total_negative_loss = tf.reduce_sum(negative_loss)

        return tf.reduce_mean(weighted_loss)
    def prototype_loss(self,prototypes_output):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        labels = tf.constant([[0], [1], [0], [1], [0], [1], [0], [1], [0], [1]], dtype=tf.float32)
        loss = bce(labels, prototypes_output)
        return loss
    
    def call(self, x, training=None):

        vec,duration,label = x
        #vec = x
        #tf.print(vec.shape)
        #tf.print(duration.shape)
        #tf.print(label.shape)
        a = tf.expand_dims(vec, -2)

        b = self.prototypes
        n_batch = tf.shape(vec)[0]

        dot_product = tf.multiply(a, b)

        norm_a = tf.norm(a, axis=-1, keepdims=True)  # 保持维度，使得维度是 [batch_size, 1, 1]

        norm_b = tf.norm(b, axis=-1, keepdims=True)  # 保持维度，使得维度是 [1, k, 1]

        # (batch_size, k,1)
        sum_product = tf.reduce_sum(dot_product, axis=-1, keepdims=True)
        cos_similarity = sum_product / (norm_a * norm_b)
        
        #tf.print(cos_similarity.shape)
        """
        tf.print(vec,summarize=-1)
        tf.print(self.prototypes,summarize=-1)
        tf.print("Input duration")
        tf.print(duration.shape)
        tf.print("Input label")
        tf.print(label.shape)
        #tf.print(vec)        """"""

        tf.print(norm_a.shape)
        tf.print(norm_a)

        tf.print(norm_b.shape)
        tf.print(norm_b)

        tf.print(cos_similarity, summarize=-1)
        """
        # （batch_size, k//2, 2, 1)
        reshaped_sum_product = tf.reshape(cos_similarity, (n_batch, self.k // 2, 2))

        temperature = 0.05

        scaled_logits = reshaped_sum_product / temperature

        softmaxed_sum_product = tf.nn.softmax(scaled_logits, axis=2)

        # (batch_size, k,1)
        alpha_product = tf.reshape(softmaxed_sum_product, (n_batch, self.k, 1))
        #tf.print(alpha_product,summarize=-1)
        """
        tf.print(alpha_product.shape)
        tf.print(alpha_product)
        """
        #self.new = alpha_product
        # n_batch, self.k, 64
        # b:(1, k, 64)
        alpha_product_expanded = tf.broadcast_to(alpha_product, [n_batch, self.k, 576])

        # calculate
        weighted_products = alpha_product_expanded * b

        # add up
        reshaped_weighted_products = tf.reshape(weighted_products, [n_batch, self.k//2, 2, 576])

        # (batchsize,5,64)
        pre_output = tf.reduce_sum(reshaped_weighted_products, axis=2)  # 对第三个维度求和

        # 1 -> [0,1,0,0,0]
        duration_one_hot = tf.one_hot(duration, depth=5)
        duration_one_hot_expanded = tf.expand_dims(duration_one_hot, -1)
        #(batch_size, 5, 1)
        duration_one_hot_expanded = tf.squeeze(duration_one_hot_expanded, axis=[1])

        masked_output = pre_output * duration_one_hot_expanded
        # output (batchsize,64)
        output = tf.reduce_sum(masked_output, axis=1)


        # get_alpha
        alpha_neg, alpha_pos = self.get_alpha(n_batch,duration_one_hot_expanded,alpha_product)
        #tf.print(alpha_pos,summarize = -1)
        #tf.print(alpha_neg,summarize = -1)
        #tf.print(alpha_pos,summarize   =-1)
        #tf.print(label, summarize=-1)
        classifier_outputs = []


        
        for i in range(5):
            classifier_output = self.binary_classifiers[i](tf.squeeze(self.prototypes,axis=0))
             # (batch_size, 1, 1)#
            #classifier_output += i
            classifier_outputs.append(classifier_output)
        
        #tf.print(classifier_outputs,summarize = -1)
        #tf.print(classifier_outputs.shape,summarize = -1)
        indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  
        prototype_output_elements = []

        for i, idx in enumerate(indices):
            tensor_index = i // 2  
            element = tf.gather(classifier_outputs[tensor_index], idx, axis=0) 
            prototype_output_elements.append(element)

        prototype_output = tf.concat(prototype_output_elements, axis=0)
        prototype_output = tf.expand_dims(prototype_output,axis=1)
        #(1,10)
        #tf.print(prototype_output)

        positive_indices = 2 * duration + 1
        negative_indices = 2 * duration

        positive_indices = tf.clip_by_value(positive_indices, 0, prototype_output.shape[0] - 1)
        negative_indices = tf.clip_by_value(negative_indices, 0, prototype_output.shape[0] - 1)
        #tf.print(alpha_pos,summarize = -1)
        # 提取对应的 prototype_output 值
        positive_values = tf.gather(prototype_output, positive_indices)
        negative_values = tf.gather(prototype_output, negative_indices)
        #positive_values = tf.squeeze(positive_values,axis=0)
        #negative_values = tf.squeeze(negative_values,axis=0)
        output = alpha_pos * tf.squeeze(positive_values,axis=-1) 
        assign_loss = self.assign_loss(output,vec)
        #tf.print(label,summarize = -1)
        #tf.print(alpha_pos,summarize = -1)
        new = self.binary_loss(label,output)
        assign = self.binary_loss(label,alpha_pos)
        self.add_loss(0.1 * assign)

        ortho_loss = self.orthogonality_loss(self.prototypes,self.k)
        self.add_loss(0.3 * ortho_loss)

        contrastive_loss = self.contrastive_loss(vec,label,duration)
        self.add_loss(0.12 * contrastive_loss)

        proto_loss = self.prototype_loss(prototype_output)
        self.add_loss(0.1 * proto_loss)
        #tf.print(new)
        #tf.print(assign)
        #tf.print(ortho_loss)
        #tf.print(contrastive_loss)
        #tf.print(proto_loss)
        return output

In [17]:
from tensorflow.keras.layers import Input, Dense,Lambda
from tensorflow.keras.models import Model
base_model_backbone = DeepCrossNet(feature_fields=fields, embed_dim=64, num_layers=3, mlp_dims=[256, 128,64], dropout=0.5)
inputs = Input(shape=(9,), dtype='int32')
duration_id_input = Lambda(lambda x: x[:, 2:3], name='duration_id_extractor')(inputs)
labels = Lambda(lambda x: x[:, -1:], name='labels')(inputs)

# 将输入层传递给自定义模型
outputs = base_model_backbone(inputs)



In [18]:
outputs

<KerasTensor: shape=(None, 576) dtype=float32 (created by layer 'deep_cross_net_1')>

In [19]:
proto_tensor = tf.random.normal([10, 576], mean=0.0, stddev=1.0, dtype=tf.float32)

In [20]:
from tensorflow.keras.models import Model
seed_tensorflow(seed=para['seed'])

flag = 1
if flag:
    prototypes_layer = Prototypes(k=10,init_prototypes = proto_tensor
                                  )

    output = prototypes_layer([outputs,duration_id_input,labels])
    #output = prototypes_layer([outputs])
    #output.shape
else:
    output = tf.keras.layers.Dense(1, activation='sigmoid')(backbone.outputs[0])


model = tf.keras.Model(inputs=inputs, outputs=[output])

seed_tensorflow(seed=para['seed'])

adam=tf.keras.optimizers.Adam(learning_rate=para['lr'])
model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]  # 确保度量名称与回调中使用的名称匹配
)




2024-07-28 14:21:03.990092: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:03.990300: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:03.990426: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [21]:
seed_tensorflow(seed=para['seed'])

adam=tf.keras.optimizers.Adam(learning_rate=para['lr'])

model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['AUC'])

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # 监控验证集损失
    mode='min',          # 最小化验证集损失
    patience=para['callback']['patience']  # 等待的轮数
)

checkpoint_auc = tf.keras.callbacks.ModelCheckpoint(
    filepath=para['callback']['CPF_model'],  # 保存模型的路径
    monitor='val_auc',  # 监控验证集 AUC
    mode='max',          # 保存验证 AUC 最大的模型
    save_weights_only=True,  # 只保存模型权重
    save_best_only=True  # 只保存最佳模型
)


hist = model.fit(train_X,
                 train_y,
                 epochs=200,
                 batch_size=512,
                 shuffle=True,
                 verbose=para['verbose'],
                 callbacks=[checkpoint_auc,es_callback],
                 validation_data=(valid_X,valid_y))

Epoch 1/200


2024-07-28 14:21:04.354197: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:04.354364: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:21:04.354434: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1431/1431 [==============================] - 24s 15ms/step - loss: 2.1882 - auc: 0.6260 - val_loss: 2.1265 - val_auc: 0.6718
Epoch 2/200
1431/1431 [==============================] - 13s 9ms/step - loss: 2.1325 - auc: 0.6669 - val_loss: 2.0969 - val_auc: 0.7001
Epoch 3/200
1431/1431 [==============================] - 13s 9ms/step - loss: 2.1119 - auc: 0.6907 - val_loss: 2.0813 - val_auc: 0.7199
Epoch 4/200
1431/1431 [==============================] - 13s 9ms/step - loss: 2.0976 - auc: 0.7103 - val_loss: 2.0706 - val_auc: 0.7340
Epoch 5/200
1431/1431 [==============================] - 10s 7ms/step - loss: 2.0867 - auc: 0.7251 - val_loss: 2.0629 - val_auc: 0.7437
Epoch 6/200
1431/1431 [==============================] - 10s 7ms/step - loss: 2.0785 - auc: 0.7358 - val_loss: 2.0571 - val_auc: 0.7502
Epoch 7/200
1431/1431 [==============================] - 14s 9ms/step - loss: 2.0717 - auc: 0.7439 - val_loss: 2.0531 - val_auc: 0.7541
Epoch 8/200
1431/1431 [==============================] - 15

In [22]:

# infer
"""
0.78959733
0.46864867
"""

seed_tensorflow(seed=para['seed'])
model.load_weights(para['callback']['CPF_model'])
y_test_pred = model.predict(test_X,batch_size=10000)
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[0], y_test_pred)
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[0], y_test_pred)
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

2024-07-28 14:33:50.869484: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:33:50.869783: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 14:33:50.869882: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

26/26 [==============================] - 0s 6ms/step
auc: 0.7879866
logloss: 0.47016987
